In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from polimi.systems import vdp, vdp_jac
import polimi.envelope as envelope

In [2]:
A = [0,0]
T = [10,100]
epsilon = 0.001
#y0 = np.random.uniform(size=2)
y0 = [2.00628952e-03, 6.23416249e-19]
fun = lambda t,y: vdp(t,y,epsilon,A,T)
jac = lambda t,y: vdp_jac(t,y,epsilon)
ttran = 0
if ttran > 0:
    print('Integrating the transient...',end='')
    tran = solve_ivp(fun,[0,ttran],y0,method='BDF',
                     atol=1e-8+np.zeros(2),
                     rtol=1e-10,jac=jac)
    print(' done.')
    plt.plot(tran['t'],tran['y'][0],'k')
    plt.show()
    y0 = tran['y'][:,-1]

In [4]:
G,period = envelope._one_period(fun, 2*np.pi, y0, 2*np.pi, jac)
print('The period is %.6f.' % period)
print('G =', G)

> /Users/daniele/PoliMi/Research/Numerical_integration/polimi/envelope.py(29)_one_period()
     28     # integrate the system for 2 guesses of the period
---> 29     if np.isscalar(atol):
     30         atol += np.zeros(len(y0))

ipdb> p t0
6.283185307179586
ipdb> p y0
[0.00200628952, 6.23416249e-19]
ipdb> p w
array([ 3.1073095e-16, -1.0000000e+00])
ipdb> p b
-0.0
ipdb> p f
array([ 6.23416249e-19, -2.00628952e-03])
ipdb> c
The period is 6.283186.
G = [1.00415758e-06 1.55300508e-19]


In [10]:
t0 = 6.283185861831177
y0 = np.array([2.00628952e-03, 6.23416249e-19])
T_guess = 6.283185861831177
atol = np.array([1.e-10, 1.e-10])
rtol = 1e-08
w = np.array([3.1073095e-16, -1.0000000e+00])
b = -0.0
G,period = envelope._one_period(fun, t0, y0, T_guess, jac, rtol, atol)

w = [ 3.1073095e-16 -1.0000000e+00]
b = -0.0


In [8]:
print(period)

6.283185861829262
